# Univariate Models

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

from src.data_preprocessing.data_loader import load_data, time_split
from src.modeling.evaluation import smape, mae
from src.modeling.univariate_modeling import get_best_cv_model

In [ ]:
DATA_DIR = (
    Path("..")
    / ".."
    / ".."
    / "hfactory_magic_folders"
    / "plastic_cost_prediction"
    / "data"
)
MAIN_FILE = "PA6_cleaned_dataset.csv"

In [ ]:
df = load_data(DATA_DIR / MAIN_FILE)

In [ ]:
models = get_best_cv_model(df)

In [ ]:
# define colors for each model type
color_dict = {
    "baseline": "grey",
    "ARIMA": "orange",
    "ETS": "blue",
    "XGB": "green",
}

for col in models:
    # get series of each col, where last 9 months are test values
    series = df[col].copy().dropna()
    y_train = series.iloc[:-9]
    y_test = series.iloc[-9:]

    # create figure for time series plot of this column
    plt.figure(figsize=(10, 6))
    plt.plot(y_train.index, y_train, label="Training data", color="black")
    plt.plot(y_test.index, y_test, label="Test data", color="red")

    for model in models[col]:
        if model not in ["selected", "XGB"]:
            # plot line graph for recursive and baseline method
            plt.plot(
                y_test.index,
                models[col][model]["preds"],
                label=model,
                color=color_dict[model],
            )
        elif model == "XGB":
            # for direct model plot points to show that it is not
            # continous and different models
            plt.plot(
                y_test.index[2::3],
                models[col][model]["preds"],
                ".",
                label=model,
                color=color_dict[model],
            )

    # add plot information
    plt.xlabel("Time")
    plt.ylabel("Values")
    plt.legend()
    plt.title(
        f"{col} - Actual vs Predictions \n Final Model: {models[col]['selected']}"
    )
    plt.show()

## Best Univariate Model for the Target

We want to invesigate the average performance across folds for each time horizon (3-6-9 months) of the best model as well as look at the residuals of each fold. 

Note: The time horizon is defined by the goal of the project, i.e. giving predictions for 3, 6, and 9 months into the future.

In [ ]:
target = "best_price_compound"

# define future index
idx = pd.DatetimeIndex(
    [
        "2023-02-01",
        "2023-03-01",
        "2023-04-01",
        "2023-05-01",
        "2023-06-01",
        "2023-07-01",
        "2023-08-01",
        "2023-09-01",
        "2023-10-01",
    ]
)

In [ ]:
# get time series for target and cv-splits as well as best model
series = df[target].copy().dropna()
target_model = models[target][models[target]["selected"]]
spl = time_split(series)

# create lists for target/preds for 3, 6, and 9 months for each fold
preds_3, preds_6, preds_9 = [], [], []
target_3, target_6, target_9 = [], [], []

# create empty list for residuals
res = []

# create figure for residuals of each cv
fig, ax = plt.subplots(figsize=(10, 4))
plt.plot(series.index, series, label="Target", color="blue")
plt.title("Target vs. Predictions per Fold")
plt.xlabel("Time")
plt.ylabel("Best Compound Price")

for i, (train_idx, test_idx) in enumerate(spl):
    # create train and test data and append target for fold fold
    train = series.iloc[train_idx]
    test = series.iloc[test_idx]
    target_3.append(test[2])
    target_6.append(test[5])
    target_9.append(test[8])

    # train model for fold
    model = ARIMA(
        train,
        order=target_model["order"],
        seasonal_order=target_model["seasonal_order"],
    )
    model_fit = model.fit()

    # use model to predict on test index
    # append predictions for 3, 6, and 9 months for fold
    preds = model_fit.predict(start=test.index[0], end=test.index[-1])
    preds_3.append(preds[2])
    preds_6.append(preds[5])
    preds_9.append(preds[8])

    if i == 0:
        plt.plot(test.index, preds, label="Validation", color="orange")
    else:
        plt.plot(test.index, preds, color="orange")
    plt.plot(test.index[2::3], preds[2::3], ".", color="orange")

    # calculate residuals for all test data
    res.append(test - preds)

# train model on all data and add predictions to plot
model = ARIMA(
    series,
    order=target_model["order"],
    seasonal_order=target_model["seasonal_order"],
)
model_fit = model.fit()
preds = model_fit.predict(start=idx[0], end=idx[-1])

# plot predictions for future
plt.plot(idx, preds, color="black", label="Prediction")
plt.plot(idx[2::3], preds[2::3], ".", color="black")

# add confidence bounds for future predictions
conf = model_fit.get_forecast(steps=9).summary_frame()
ax.fill_between(
    conf.index,
    conf["mean_ci_lower"],
    conf["mean_ci_upper"],
    color="k",
    alpha=0.1,
)

plt.legend(loc="upper left")

In [ ]:
# calculate sMAPE for 3, 6, and 9 months
smape_3 = smape(np.array(target_3), np.array(preds_3))
smape_6 = smape(np.array(target_6), np.array(preds_6))
smape_9 = smape(np.array(target_9), np.array(preds_9))

# calculate MAE for 3, 6, and 9 months
mae_3 = mae(np.array(target_3), np.array(preds_3))
mae_6 = mae(np.array(target_6), np.array(preds_6))
mae_9 = mae(np.array(target_9), np.array(preds_9))

In [ ]:
print("** Fold-averaged results for time horizons **")
print("3 months:\n", f"- sMAPE: {smape_3:.2f}% \n", f"- MAE: {mae_3:.2f}")
print("6 months:\n", f"- sMAPE: {smape_6:.2f}% \n", f"- MAE: {mae_6:.2f}")
print("9 months:\n", f"- sMAPE: {smape_9:.2f}% \n", f"- MAE: {mae_9:.2f}")

In [ ]:
# plot residuals
plt.figure(figsize=(10, 4))
plt.title("Residuals")
plt.axhline(
    0, color="red", linestyle="--", linewidth=2, label="Zero Residuals"
)
for residual in res:
    plt.plot(residual.index, residual, label="Residuals", color="blue")

## Feature Importance

Get the feature importance of the best univariate model on the last fold (most train data) for explainability.

In [ ]:
print(model_fit.summary())

## CV Folds for Explainability

In [ ]:
# get time series for target and cv-splits
series = df[target].copy().dropna()
spl = time_split(series)

# create figure for residuals of each cv
fig, ax = plt.subplots(figsize=(13, 3))
plt.plot(series.index, series, label="Target", color="black")
plt.xlabel("Time")
plt.ylabel("Best Compound Price")

for i, (train_idx, test_idx) in enumerate(spl):
    # create train and test data and append target for fold fold
    train = series.iloc[train_idx]
    test = series.iloc[test_idx]

    plt.axvline(test.index[0], linestyle="--", color="grey")
    plt.text(
        test.index[2],
        3600,
        "fold %d" % (i + 1),
        verticalalignment="center",
        color="grey",
    )